# Learning Database Questions

This notebook collects questions to pose to the Learning database and the queries to answer them.

**How many students have declared for each program combination per institution (i.e. major-major, major-minor, major-only)?**

In [1]:
USE Learning;
GO

WITH StudentProgramCount (InstitutionKey, StudentKey, MajorCount, MinorCount)
AS
(
	SELECT
		InstitutionKey			AS InstitutionKey,
		StudentKey				AS StudentKey,
		Major					AS MajorCount,
		Minor					AS MinorCount
	FROM
	(
		SELECT
			sp.InstitutionKey,
			sp.StudentKey,
			p.ProgramType
		FROM
			Enrollment.StudentProgram sp
		INNER JOIN
			Curriculum.Program p
		ON
			p.InstitutionKey = sp.InstitutionKey
		AND
			p.DepartmentKey = sp.DepartmentKey
		AND
			p.ProgramKey = sp.ProgramKey
	) AS StudentProgramType
	PIVOT
	(
		COUNT(ProgramType)
		FOR ProgramType in (Major, Minor)
	) AS StudentProgramTypePivot
)
SELECT
	spc.InstitutionKey			AS InstitutionKey,
	i.DisplayName				AS InstitutionName,
	spc.MajorCount				AS MajorCount,
	spc.MinorCount				AS MinorCount,
	COUNT(*)					AS Students
FROM
	StudentProgramCount spc
INNER JOIN
	Organization.Institution i
ON
	i.InstitutionKey = spc.InstitutionKey
INNER JOIN
	Enrollment.Student s
ON
	s.InstitutionKey = spc.InstitutionKey
AND
	s.StudentKey = spc.StudentKey
GROUP BY
	spc.InstitutionKey,
	i.DisplayName,
	spc.MajorCount,
	spc.MinorCount
ORDER BY
	i.DisplayName,
	spc.InstitutionKey,
	spc.MajorCount DESC,
	spc.MinorCount DESC;
GO


Commands completed successfully.

(3 rows affected)

Total execution time: 00:00:00.088

InstitutionKey,InstitutionName,MajorCount,MinorCount,Students
5c06fc60-fe37-43c5-84a2-a2c0d4024ef6,Langworth - Brakus,2,0,2993
5c06fc60-fe37-43c5-84a2-a2c0d4024ef6,Langworth - Brakus,1,1,2936
5c06fc60-fe37-43c5-84a2-a2c0d4024ef6,Langworth - Brakus,1,0,3071


> Not Started

**What is the student's average score for their courses?**

- Only program courses?
- Only non-program courses?

> In Progress

**Has the student met the requirements for their declared programs?**
* Passing score in all required classes.
* The model ignores general education credit requirements.
* The current data generation strategy makes it unlikely for any student to meet their program requirements.

In [1]:
USE Learning;
GO

DECLARE @InstitutionKey		uniqueidentifier	= (SELECT TOP 1 InstitutionKey FROM Organization.Institution TABLESAMPLE (100 PERCENT));
DECLARE @StudentKey			uniqueidentifier	= (SELECT TOP 1 StudentKey FROM Enrollment.Student TABLESAMPLE (10 PERCENT) WHERE InstitutionKey = @InstitutionKey);

IF @InstitutionKey IS NULL
	THROW 50000, N'No sample taken for InstitutionKey.', 10;

IF @StudentKey IS NULL
	THROW 50000, N'No sample taken for StudentKey.', 20;

PRINT N'Institution Key:  ' + COALESCE(CAST(@InstitutionKey AS nvarchar(36)), N'No key sampled.');
PRINT N'Student Key:      ' + COALESCE(CAST(@StudentKey AS nvarchar(36)), N'No key sampled.');

-- Display the courses for the programs declared by the student
-- with associated enrollment information.
SELECT
	s.DisplayName								AS StudentName,
	i.DisplayName								AS InstitutionName,
	d.DisplayName								AS DepartmentName,
	p.DisplayName								AS ProgramName,
	p.ProgramType								AS ProgramType,
	c.DisplayName								AS CourseName,
	c.Level										AS CourseLevel,
	t.CalendarYear								AS TermYear,
	t.SeasonName								AS TermSeason,
	coe.Score									AS Score
FROM
	Enrollment.StudentProgram sp
INNER JOIN
	Organization.Institution i
ON
	i.InstitutionKey = sp.InstitutionKey
INNER JOIN
	Enrollment.Student s
ON
	s.InstitutionKey = sp.InstitutionKey
AND
	s.StudentKey = sp.StudentKey
INNER JOIN
	Organization.Department d
ON
	d.InstitutionKey = sp.InstitutionKey
AND
	d.DepartmentKey = sp.DepartmentKey
INNER JOIN
	Curriculum.Program p
ON
	p.InstitutionKey = sp.InstitutionKey
AND
	p.DepartmentKey = sp.DepartmentKey
AND
	p.ProgramKey = sp.ProgramKey
INNER JOIN
	Curriculum.ProgramCourse pc
ON
	pc.InstitutionKey = p.InstitutionKey
AND
	pc.DepartmentKey = p.DepartmentKey
AND
	pc.ProgramKey = p.ProgramKey
INNER JOIN
	Curriculum.Course c
ON
	c.InstitutionKey = pc.InstitutionKey
AND
	c.DepartmentKey = pc.DepartmentKey
AND
	c.CourseKey = pc.CourseKey
LEFT OUTER JOIN
	Enrollment.CourseOffering co

	INNER JOIN
		Enrollment.CourseOfferingEnrollment coe
	ON
		coe.InstitutionKey = @Institutionkey
	AND
		coe.CourseOfferingKey = co.CourseOfferingKey
	AND
		coe.StudentKey = @StudentKey
	INNER JOIN
		Enrollment.Term t
	ON
		t.InstitutionKey = co.InstitutionKey
	AND
		t.TermKey = co.TermKey
ON
	co.InstitutionKey = c.InstitutionKey
AND
	co.DepartmentKey = c.DepartmentKey
AND
	co.CourseKey = c.CourseKey
WHERE
	s.InstitutionKey = @InstitutionKey
AND
	s.StudentKey = @StudentKey
ORDER BY
	d.DisplayName,
	p.ProgramType,
	p.DisplayName,
	c.Level,
	c.DisplayName;
GO


Commands completed successfully.

Institution Key: 5C06FC60-FE37-43C5-84A2-A2C0D4024EF6

Student Key: F019029C-6433-4DE2-A239-B95EFD12CA34

(24 rows affected)

Total execution time: 00:00:00.085

StudentName,InstitutionName,DepartmentName,ProgramName,ProgramType,CourseName,CourseLevel,TermYear,TermSeason,Score
Helga Hahn,Langworth - Brakus,Dept. of Functionality,Drive Science,Major,Navigating Multi-Byte Sensor,129,2018,Spring,97
Helga Hahn,Langworth - Brakus,Dept. of Functionality,Drive Science,Major,Programming Bluetooth Transmitter,135,NULL,NULL,NULL
Helga Hahn,Langworth - Brakus,Dept. of Functionality,Drive Science,Major,Transmitting Multi-Byte Matrix,257,NULL,NULL,NULL
Helga Hahn,Langworth - Brakus,Dept. of Functionality,Drive Science,Major,Generating Auxiliary Hard Drive,264,NULL,NULL,NULL
Helga Hahn,Langworth - Brakus,Dept. of Functionality,Drive Science,Major,Transmitting Digital Panel,321,NULL,NULL,NULL
Helga Hahn,Langworth - Brakus,Dept. of Functionality,Drive Science,Major,Copying Redundant Monitor,365,NULL,NULL,NULL
Helga Hahn,Langworth - Brakus,Dept. of Functionality,Drive Science,Major,Hacking Auxiliary Bandwidth,372,2015,Fall,95
Helga Hahn,Langworth - Brakus,Dept. of Functionality,Drive Science,Major,Navigating Online Circuit,391,2018,Spring,82
Helga Hahn,Langworth - Brakus,Dept. of Functionality,Drive Science,Major,Navigating 1080P Feed,399,NULL,NULL,NULL
Helga Hahn,Langworth - Brakus,Dept. of Functionality,Drive Science,Major,Hacking Primary Application,448,2017,Fall,75
